In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import sqlalchemy
import string

In [19]:
np.random.seed(42)
df1 = pd.DataFrame(np.random.randn(10000,5),columns=list(string.ascii_uppercase[:5]))
df1.head()
df2 = pd.DataFrame(np.random.randn(10000,5),columns=list(string.ascii_uppercase[5:10]))
df2.head()

,F,G,H,I,J
0,0.098206,-0.064108,0.951791,1.532831,0.686847
1,1.011544,-0.217404,0.125405,1.485732,0.526711
2,-0.242030,0.032551,1.037151,1.126227,-0.842131
3,0.362591,-0.007843,-0.065536,1.330308,-0.237538
4,-0.575577,-0.814991,-0.174547,-0.735246,-0.864782


In [7]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [20]:
with engine.begin() as conn:
    df1.to_sql("tn1",conn,if_exists="replace",index=False)
    df2.to_sql("tn2",conn,if_exists="replace",index=False)

2022-09-06 23:33:32,910 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-06 23:33:32,916 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tn1")
2022-09-06 23:33:32,920 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-06 23:33:32,923 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tn1")
2022-09-06 23:33:32,924 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-06 23:33:32,929 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-09-06 23:33:32,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-06 23:33:32,935 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("tn1")
2022-09-06 23:33:32,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-06 23:33:32,939 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-09-06 23:33:32,940 INFO sqlalchemy.engine.Engine [raw sql] ('tn1',)
2022-09-06 23:33:32,942 INFO sqlalche

In [21]:
with engine.connect() as conn:
    df = pd.read_sql(text(f"""
    with t as (
        select sum(A) s
        from tn1
    ), t2 as (
        select sum(F) s
        from tn2
    )
    select s
    from (
        select * from t
        union all
        select * from t2
    )
    """),conn)
df

2022-09-06 23:33:40,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-06 23:33:40,498 INFO sqlalchemy.engine.Engine 
    with t as (
        select sum(A) s
        from tn1
    ), t2 as (
        select sum(F) s
        from tn2
    )
    select s
    from (
        select * from t
        union all
        select * from t2
    )
    
2022-09-06 23:33:40,499 INFO sqlalchemy.engine.Engine [cached since 350s ago] ()
2022-09-06 23:33:40,502 INFO sqlalchemy.engine.Engine ROLLBACK


,s
0,0.022847
1,51.698964


In [29]:
list(df.index)==[0,1]

True

In [26]:
np.linalg.norm(df.values-[[2.28467531e-02],
       [5.16989643e+01]])

4.148276000713217e-08